In [ ]:
# !pip install langchain_community langchainhub chromadb langchain langchain-openai

In [1]:
!pip install langchain_community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.

In [2]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai"])

docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Learn, Build, Deploy and Apply Generative AI', 'language': 'en'}, page_content="Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 6th Feb, 2026Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · Fri, Sat and Sun · 9:00 PM - 11:00 PM IST (every class) + Post-class doubt supportLast fully LIVE batch. All future courses will follow a hybrid format!Access all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mi

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
splits = text_splitter.split_documents(docs)

In [5]:
print(splits[0])
print(splits[1])
print(splits[2])

page_content='Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 6th Feb, 2026Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · Fri, Sat and Sun · 9:00 PM - 11:00 PM IST (every class) + Post-class doubt supportLast fully LIVE batch. All future courses will follow a hybrid format!Access all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week' metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Learn, Build, Deploy and Apply Generative AI', 'language': 'en'}
page_content='RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Arc

In [6]:
print(len(splits))

39


In [7]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [8]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)

/tmp/ipython-input-2599906244.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
print(vectorstore._collection.count())

39


In [10]:
print(vectorstore._collection.get())

{'ids': ['86a97490-a311-41a1-964d-ad48014c5eca', 'c5c90d5a-f914-4985-b0d4-6db02da84b3f', '76bf09b2-8c97-4ba4-acca-d5e77c229fdd', 'f447e6c8-29f1-4754-a61b-17438dfd3396', 'c4362b1c-36ce-425e-b2cf-e7fdcebf5763', 'ef24b088-0d95-45ca-9586-f74cd9240fb2', '01c27701-c6d9-4e44-af96-ffb43c75e243', 'cb38c2be-864c-428a-8229-03ef31517c01', '22749fe1-0701-46e5-8457-55288bdfeed3', 'ff7b32ea-d86c-44ec-87a1-d849933b7df2', 'cb1844d0-534f-4e00-ab14-1b17f09a1ce0', '5e5fa0aa-55b0-412b-a152-943c7cc643dc', '70ed784e-6ea2-43a8-939c-8e82a922b897', 'b34c6e15-fb4a-4d3c-b490-f654ce967a8e', 'e6a84104-f75d-4343-9ee7-4618f901f43e', '97c16bf2-4df4-4184-a335-5b6ae9d2ccd0', '9e4faebf-1d21-4066-a740-d6e04e723a01', 'b5696449-2497-4530-a151-78d6ab334c33', 'e56825c2-da11-46b7-9531-5b97b4b3101a', 'f302ce08-a904-466d-8da7-3e1fb279e670', '402883a9-35de-421f-9dd7-5393006cc04d', '7cf031f7-9540-426e-9ee6-4890c3739c32', '96700ed3-6115-4126-990c-5e5f48014643', '446a8e31-804c-4d41-8c87-3d7634e2c5a5', '3b211bd6-5bb5-4836-8e20-6be4b9

In [11]:
print("\nCollection 1 - ", vectorstore._collection.get(ids=['86a97490-a311-41a1-964d-ad48014c5eca'], include=["embeddings", "documents"]))
print("\nCollection 2 - ", vectorstore._collection.get(ids=['c5c90d5a-f914-4985-b0d4-6db02da84b3f'], include=["embeddings", "documents"]))
print("\nCollection 3 - ", vectorstore._collection.get(ids=['76bf09b2-8c97-4ba4-acca-d5e77c229fdd'], include=["embeddings", "documents"]))


Collection 1 -  {'ids': ['86a97490-a311-41a1-964d-ad48014c5eca'], 'embeddings': array([[-7.84601197e-02, -9.95943621e-02, -7.64424214e-03,
        -4.43422422e-02, -1.38636408e-02,  1.27232112e-02,
        -1.32839372e-02, -6.39203563e-02, -8.87689218e-02,
         1.58299692e-02,  7.34179886e-03, -5.41536100e-02,
        -3.03664524e-02, -3.17796171e-02, -1.25836274e-02,
         4.67547914e-03,  3.20336409e-03, -1.43723236e-02,
         3.07857953e-02,  1.15024486e-04,  6.43830688e-04,
         1.63509157e-02,  1.25596607e-02,  2.54901741e-02,
         2.28286139e-03,  2.17974484e-02,  2.76003103e-03,
        -5.07067069e-02,  5.58193102e-02, -8.38756561e-02,
         1.28571084e-02,  1.15510590e-01,  1.63756739e-02,
         6.52803481e-02,  4.63707335e-02,  3.95750441e-02,
        -1.46374013e-02, -1.65717248e-02,  5.50052375e-02,
        -2.88963765e-02, -5.52620403e-02, -1.63694955e-02,
         6.92707160e-03, -1.20429893e-03,  4.27914374e-02,
        -3.54133639e-03, -1.109185

In [12]:
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

In [13]:
# from langchain import hub
# prompt = hub.pull("rlm/rag-prompt")

In [14]:
# The 'hub' object (for pulling prompts from LangChain Hub) is imported directly from the 'langchain_hub' package.
# However, it appears there's still an issue importing 'langchain_hub' or accessing 'hub.pull()'.
# To ensure the prompt is defined and allow the chain to run, we will explicitly define the common 'rlm/rag-prompt' template.
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."),
    ("human", "Question: {question}\nContext: {context}\nAnswer:")
])

In [15]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI()

In [16]:
# from langchain_community.llms import HuggingFaceHub
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0.5, "max_length":512}, task="text2text-generation")

from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512
)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-970550028.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [17]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [19]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [20]:
rag_chain = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [21]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

'Lifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week'

In [22]:
rag_chain.invoke("Are the testimonials for the course available? Name the studenst who have shared testimonials")

'Keerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.'

In [23]:
rag_chain.invoke("Are the certificates for the course provided?")

'Yes.'

In [24]:
rag_chain.invoke("What all projects are covered in the course?")

'Build, Deploy and Apply Generative AI7 weeks  Fri, Sat and Sun  9:00 PM - 11:00 PM IST (every class) + Post-class doubt supportLast fully LIVE batch. All future courses will follow a hybrid format!Access all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week Software EngineerOptum-United HealthGroupKeerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.'

In [ ]:
from langchain_core.runnables import RunnableLambda

In [ ]:
# def print_prompt(prompt_text):
#   print("Prompt - ", prompt_text)
#   return prompt_text

In [ ]:
# rag_chain_with_print = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
#              | prompt
#              | RunnableLambda(print_prompt)
#              | llm
#              | StrOutputParser())

In [ ]:
# rag_chain_with_print.invoke("What all projects are covered in the course?")

In [ ]:
## Testing of openai api key

In [25]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [26]:
from openai import OpenAI

# Create client (API key is read from environment variable)
client = OpenAI()

print("🤖 Simple Chatbot — type 'exit' to quit\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye! 👋")
        break

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a friendly chatbot that helps users."},
            {"role": "user", "content": user_input}
        ]
    )

    reply = response.choices[0].message.content
    print(f"Chatbot: {reply}\n")


🤖 Simple Chatbot — type 'exit' to quit

You: hii
Chatbot: Hello! How can I assist you today?

You: what are some of the latest news?
Chatbot: I don't have access to real-time news updates, but I can help guide you on where to find the latest news or provide summaries of key topics up to October 2023. For the most recent news, I recommend checking reliable news websites, apps, or social media platforms. If you’re interested in a specific topic or recent event, let me know, and I can provide more context or background information!

You: which are some of the free tier gpt models?
Chatbot: As of my last update in October 2023, several platforms offer free tier access to GPT models or similar language models. Here are a few options:

1. **OpenAI's GPT Models**: OpenAI may offer a free tier with limited usage of their models like GPT-3.5 or GPT-4 through the ChatGPT interface. Always check their website for the latest offerings and usage limits.

2. **Hugging Face**: Hugging Face provides a